In [400]:
import pandas as pd
import numpy as np
import json
from json import JSONEncoder
import glob
from pathlib import Path
from tqdm.auto import tqdm
import os
import zipfile
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [409]:
import pathMgmt as myPath

In [410]:
myPath.DATA_DIR

WindowsPath('data')

In [16]:
DATA_DIR = Path('data')

In [112]:
MAPPING = {"stk_1min": '1minProcess',
           "adj_factor": "adj_fct",
           "index": 'idx',
           "limits": "lmt",
           "mkt_value": "mkt_val",
           "industry": "sw",
           "universe": "univ"}

In [113]:
# idx
# code,name,date,pre_close,open,high,low,close,amt,vol,chg,chg_pct

In [203]:
def getfilePath(tab_name, date, indexName=None):
    if tab_name in ["index", "adj_factor", "limits", "mkt_value", "industry"]:
        return DATA_DIR / MAPPING[tab_name] / date[:4] / (date+'.csv')
    elif tab_name == "universe":
        return DATA_DIR / MAPPING[tab_name] / indexName / date[:4] / (date+'.csv')
    elif tab_name == "stk_1min":
        return DATA_DIR / MAPPING[tab_name] / (date+'.csv')

In [204]:
def loading(tab_name, start, end, fields):
    """Read data from start to end
    tab_name: table name
    start: start date
    end: end date
    fields: columns of interest
    """
    output = pd.DataFrame()
    # folderpath = getpath(tab_name)
    dates = [d.strftime("%Y%m%d")
             for d in pd.date_range(start, end)]
    for date in dates:
        filepath = getfile(
            tab_name, date, fields) if tab_name == "universe" else getfile(tab_name, date)
        if filepath.exists():
            print(filepath)
            current = pd.read_csv(
                filepath) if tab_name == "universe" else pd.read_csv(
                    filepath, usecols=fields)
            current['time'] = date
            output = pd.concat([output, current])
    output.set_index(['time'], inplace=True)
    output.index = pd.to_datetime(output.index)
    return output

In [205]:
def getDates(N, date):
    idx = bisect.bisect_left(CALENDAR, date)
    if N >= 0:
        dates = CALENDAR[idx: idx + 10]
    else:
        if date == CALENDAR[idx]:
            dates = CALENDAR[(idx + N + 1): (idx + 1)]
        else:
            dates = CALENDAR[(idx + N): idx]
    return dates

In [206]:
def loading2(tab_name, start, N, fileds):
    """Read N-days data, starting from start. 
    N can be negative, meaning reading backwardly.
    tab_name: table name
    start: start date
    N: offset, can be both positive or negative
    fields: columns of interest
    """
    output = pd.DataFrame()
    # folderpath = getpath(tab_name)
    dates = getDates(N, start)
    for date in dates:
        filepath = getfile(tab_name, date)
        if filepath.exists():
            current = pd.read_csv(filepath, usecols=fileds)
            current['time'] = date
            output = pd.concat([output, current])
    output.set_index(['time'], inplace=True)
    output.index = pd.to_datetime(output.index)
    return output

In [207]:
help(loading2)

Help on function loading2 in module __main__:

loading2(tab_name, start, N, fileds)
    Read N-days data, starting from start. 
    N can be negative, meaning reading backwardly.
    tab_name: table name
    start: start date
    N: offset, can be both positive or negative
    fields: columns of interest



In [208]:
loading("stk_1min", '20180101', '20180110', ['code','open', 'close'])

data\1minProcess\20180102.csv
data\1minProcess\20180103.csv
data\1minProcess\20180104.csv
data\1minProcess\20180105.csv
data\1minProcess\20180108.csv
data\1minProcess\20180109.csv
data\1minProcess\20180110.csv


,code,open,close
time,,,
2018-01-02,1,133500,137000
2018-01-02,2,314500,325600
2018-01-02,4,222900,223400
2018-01-02,5,41500,43200
2018-01-02,8,87500,87700
...,...,...,...
2018-01-10,603993,72400,72100
2018-01-10,603996,178500,177700
2018-01-10,603997,110000,110200


In [209]:
loading("index", '20180101', '20180110', ['code','open','close'])

data\idx\2018\20180102.csv
data\idx\2018\20180103.csv
data\idx\2018\20180104.csv
data\idx\2018\20180105.csv
data\idx\2018\20180108.csv
data\idx\2018\20180109.csv
data\idx\2018\20180110.csv


,code,open,close
time,,,
2018-01-02,1,3314.0307,3348.3259
2018-01-02,300,4045.2086,4087.4012
2018-01-02,852,7027.6346,7092.0947
2018-01-02,905,6263.1506,6332.2269
2018-01-02,906,4420.4788,4467.2791
2018-01-02,399102,2206.6290,2219.1780
2018-01-02,399106,1903.4930,1919.1980
2018-01-03,1,3347.7428,3369.1084
2018-01-03,300,4091.4607,4111.3925


In [210]:
loading("universe", '20180101', '20180110', 'hs300')

data\univ\hs300\2018\20180102.csv
data\univ\hs300\2018\20180103.csv
data\univ\hs300\2018\20180104.csv
data\univ\hs300\2018\20180105.csv
data\univ\hs300\2018\20180108.csv
data\univ\hs300\2018\20180109.csv
data\univ\hs300\2018\20180110.csv


,code,name
time,,
2018-01-02,1,平安银行
2018-01-02,2,万科A
2018-01-02,8,神州高铁
2018-01-02,60,中金岭南
2018-01-02,63,中兴通讯
...,...,...
2018-01-10,603160,汇顶科技
2018-01-10,603799,华友钴业
2018-01-10,603833,欧派家居


In [101]:
loading2("stk_1min", '20180101', 20, ['code','open', 'close'])

,code,open,close
time,,,
2018-01-02,1,133500,137000
2018-01-02,2,314500,325600
2018-01-02,4,222900,223400
2018-01-02,5,41500,43200
2018-01-02,8,87500,87700
...,...,...,...
2018-01-15,603993,78000,75100
2018-01-15,603996,179500,179600
2018-01-15,603997,110500,110300


In [51]:
CALENDAR = pd.read_csv(DATA_DIR/'date'/'trd_date.csv')

In [56]:
CALENDAR = [str(d) for d in CALENDAR.loc[CALENDAR.is_open==1].date.tolist()]

In [58]:
with open('calendar.json', 'w') as f:
  json.dump(CALENDAR, f)

In [65]:
import bisect

In [90]:
dates

['20171219',
 '20171220',
 '20171221',
 '20171222',
 '20171225',
 '20171226',
 '20171227',
 '20171228',
 '20171229',
 '20180102']

In [75]:
CALENDAR[6613]

'20180102'

In [ ]:
class data_loader():
    def __init__(self):
        

In [111]:
# process date data

In [115]:
# halt date
halt_date = pd.read_csv(DATA_DIR / 'date' / 'halt_date.csv')


In [118]:
halt_date.halt_time.value_counts()


93000     14367
130000      470
93100        79
93200        31
103000       22
          ...  
143032        1
141944        1
145557        1
95635         1
102900        1
Name: halt_time, Length: 270, dtype: int64

In [123]:
halt_date.restart_time.value_counts()

93000     13824
100000      766
100100      115
130000       79
100001       73
          ...  
143020        1
144055        1
95302         1
134836        1
103900        1
Name: restart_time, Length: 253, dtype: int64

In [124]:
# from start to end, halted stock on each day (whole day halted or part of the day halted)
halt_date


,code,name,halt_date,halt_time,restart_date,restart_time
0,1,平安银行,20140715,93000,20140716,93000
1,2,万科Ａ,20170714,130000,20170718,93000
2,2,万科Ａ,20170607,93000,20170612,93000
3,2,万科Ａ,20170112,93000,20170113,93000
4,2,万科Ａ,20151218,130000,20160704,93000
...,...,...,...,...,...,...
15362,688690,N纳微,20210623,142300,20210623,143300
15363,688711,宏微科技,20210902,140500,20210902,141500
15364,688768,N容知,20210726,102900,20210726,103900
15365,688800,N瑞可达,20210722,93100,20210722,94100


In [211]:
calendar_set = set(CALENDAR)

In [ ]:
halt_date.


In [217]:
result = []
for row in halt_date.values:
    code = row[0]
    name = row[1]
    start = str(row[2])
    end = str(row[4])
    dates = pd.date_range(start, end)
    # print(dates)
    if row[5] == 93000:
        dates = dates[:-1]
    dates = [d.strftime("%Y%m%d") for d in dates]
    # print(dates)
    for d in dates:
        result.append([d, code, name])
    # print(row)

In [219]:
df = pd.DataFrame(data=result, columns=['time', 'code', 'name'])

In [222]:
df.sort_values(by=['time', 'code']).drop_duplicates()

,time,code,name
53906,20070430,693,S*ST聚友
53907,20070501,693,S*ST聚友
53908,20070502,693,S*ST聚友
53909,20070503,693,S*ST聚友
53910,20070504,693,S*ST聚友
...,...,...,...
338468,20220207,300757,罗博特科
477019,20220207,603322,超讯通信
338469,20220208,300757,罗博特科
477058,20220208,603322,超讯通信


In [ ]:
def load_from_single(filename):
    # df =  filename
    

In [223]:
# lst date
lst_date = pd.read_csv(DATA_DIR / 'date' / 'lst_date.csv')


In [225]:
lst_date.lst_status.unique()

array(['L', 'DE', 'S', 'O', 'UN'], dtype=object)

In [407]:
def preprocessST(inputfile, outputfile):
    st_date = pd.read_csv(DATA_DIR / 'date' / inputfile)
    st_date = st_date.sort_values(by=['code', 'eff_date'])
    st_date['eff_date'] = st_date['eff_date'].astype('string')
    st_date['status_id'] = st_date['status_id'].astype('string')

    # we print out the status transition for each stock
    st_trainsition = st_date.sort_values(by=['code', 'eff_date']).groupby(
        'code').status_id.apply(lambda x: '->'.join(x))

    # print(st_trainsition.value_counts())
    # 2->1                                  312
    # 2                                     111
    # 2->1->2->1                             83
    # 2->4->3->1                             48
    # 2->4->5                                47
    # 2->1->2                                44
    # 5                                      36
    # 2->1->2->1->2->1                       18
    # 2->4->1->3                             12
    # 2->1->2->1->2                          10
    # 2->1->2->4->3->1                        8
    # 2->1->6->7->5                           6
    # 4->6->7->5                              6
    # 2->4->3->1->2->1                        6
    # 2->1->5                                 6
    # 2->4->1->6->7->5                        5
    # 2->4->6->1->7->5                        4
    # 2->4->3->1->2                           4
    # 2->1->2->4->1->3                        4
    # 2->5                                    4
    # 2->1->2->1->2->4->1->6->7->5            3
    # 2->6->1->7->5                           3
    # 2->4->3->1->2->1->2                     3
    # 2->1->2->6->1->7->5                     3
    # 2->4->3->1->5                           3
    # 2->1->2->1->2->1->2->1                  3
    # 2->1->2->4->1->6->7->5                  3
    # 6->7->5                                 3
    # 2->1->2->4->6->1->7->5                  2
    # 2->1->2->4->5                           2
    # 2->1->2->1->5                           2
    # 2->4->3->1->2->4->1->6->7->5            2
    # 2->1->2->4->3->1->2                     2
    # 2->4->2->3->1                           2
    # 2->1->2->1->2->1->2->4->3->1            2
    # 2->1->2->1->6->7->5                     2
    # 2->4->3->1->2->4->1->3                  1
    # 2->1->2->4->3->1->2->1->2               1
    # 2->4->3->1->2->4->6->1->7->5            1
    # 2->4->3->4->3->1->2->1                  1
    # 2->4->1->3->2->1->2->1->2               1
    # 2->4->3->1->2->1->2->4->1->6->7->5      1
    # 2->1->2->4->3->1->2->1                  1
    # 2->4->3->1->2->4->3->1->2               1
    # 2->4->3->4->3->1                        1
    # 2->4->1->3->2->4->6->1->7->5            1
    # 2->4->3->1->2->4->3->1                  1
    # 2->4->2->3->1->2                        1
    # 2->4                                    1
    # 2->1->2->1->2->5                        1
    # 2->1->2->4->3->1->2->1->2->4            1
    # 2->1->2->1->2->6->1->7->5               1
    # 2->4->3->2->1->2->1->2                  1
    # 2->4->3->1->2->1->2->1->2               1
    # 2->4->2->3->1->2->1                     1
    # 2->4->3->2->1->2->1                     1
    # 1                                       1
    # Name: status_id, dtype: int64

    st_date['transition'] = st_date.sort_values(by=['code', 'eff_date']).groupby(
        'code').status_id.transform(lambda x: '->'.join(x))
    
    CALENDAR_set = set(CALENDAR)
    def resample(df):
        df.drop_duplicates(subset=['code', 'eff_date'], inplace=True)
        df['eff_date'] = pd.to_datetime(df['eff_date'])
        df = df.set_index('eff_date').groupby(
            'code').apply(lambda x: x.resample('D').ffill()).reset_index(1).reset_index(drop=True)
        df['date'] = df.eff_date.dt.strftime("%Y%m%d")
        df = df.loc[df.date.transform(lambda x: x in CALENDAR_set)]
        df.drop(columns=['eff_date'], inplace=True)
        return df

    # we treat two cases separately
    # the final staus is 1 or 3
    def case1(df):
        df = df.loc[df.transition.str.endswith(
            '1') | df.transition.str.endswith('3')]
        df = resample(df)
        return df
    st_date1 = case1(st_date)
    
    # the final staus is 2 or 4 or 5
    def case2(df):
        max_date = df.eff_date.max()
        df = df.loc[df.transition.str.endswith('2')
                               | df.transition.str.endswith('4')
                               | df.transition.str.endswith('5')]
        # we get last status for each stock
        df_last_day = df.drop_duplicates(
            subset=['code'], keep='last').copy()
        # we add a row here which indicates that the status lasts to at least the largest date in the table
        df_last_day['eff_date'] = max_date
        df = pd.concat([df, df_last_day]
                             ).drop_duplicates().sort_values(by=['code', 'eff_date'])

        df = resample(df)
        return df
    st_date2 = case2(st_date)
    
    outFolder = DATA_DIR/'dateProcess'
    outFolder.mkdir(parents=True, exist_ok=True)
    output = pd.concat([st_date1, st_date2]).sort_values(by=['date'])
    output.to_csv(outFolder/outputfile, index=False)
    

In [408]:
preprocessST('st_date.csv', 'ST_date.csv')

In [404]:
result.to_csv('ST_date.csv', index=False)

In [308]:
# st date
st_date = pd.read_csv(DATA_DIR / 'date' / 'st_date.csv')
st_date = st_date.sort_values(by=['code', 'eff_date'])
st_date['eff_date'] = st_date['eff_date'].astype('string')
st_date['status_id'] = st_date['status_id'].astype('string')

In [ ]:
st_trainsition = st_date.sort_values(by=['code', 'eff_date']).groupby(
    'code').status_id.apply(lambda x: '->'.join(x))

print(st_trainsition.value_counts())

In [312]:
st_date['transition'] = st_date.sort_values(by=['code', 'eff_date']).groupby(
    'code').status_id.transform(lambda x: '->'.join(x))

In [320]:
# end with 1 or 3
st_date1 = st_date.loc[st_date.transition.str.endswith(
    '1') | st_date.transition.str.endswith('3')]

In [ ]:
st_date1.drop_duplicates(subset=['code', 'eff_date'], inplace=True)
st_date1['eff_date'] = pd.to_datetime(st_date1['eff_date'])
st_date1 = st_date1.set_index('eff_date').groupby(
    'code').apply(lambda x: x.resample('D').ffill()).reset_index(1).reset_index(drop=True)

In [352]:
st_date1.loc[st_date1.status_id != 1]

,eff_date,code,name,status_id,transition
0,1999-04-27,4,ST深安达,2,2->1->2->1
1,1999-04-28,4,ST深安达,2,2->1->2->1
2,1999-04-29,4,ST深安达,2,2->1->2->1
3,1999-04-30,4,ST深安达,2,2->1->2->1
4,1999-05-01,4,ST深安达,2,2->1->2->1
...,...,...,...,...,...
866556,2019-11-10,603810,ST丰山,2,2->1
866557,2019-11-11,603810,ST丰山,2,2->1
866558,2019-11-12,603810,ST丰山,2,2->1
866559,2019-11-13,603810,ST丰山,2,2->1


In [360]:
st_date1['date'] = st_date1.eff_date.dt.strftime("%Y%m%d")

In [375]:
CALENDAR_set = set(CALENDAR)
max_date = st_date.eff_date.max()


In [367]:
st_date1 = st_date1.loc[st_date1.date.transform(lambda x: x in CALENDAR_set)]

,eff_date,code,name,status_id,transition,date
0,1999-04-27,4,ST深安达,2,2->1->2->1,19990427
1,1999-04-28,4,ST深安达,2,2->1->2->1,19990428
2,1999-04-29,4,ST深安达,2,2->1->2->1,19990429
3,1999-04-30,4,ST深安达,2,2->1->2->1,19990430
7,1999-05-04,4,ST深安达,2,2->1->2->1,19990504
...,...,...,...,...,...,...
866554,2019-11-08,603810,ST丰山,2,2->1,20191108
866557,2019-11-11,603810,ST丰山,2,2->1,20191111
866558,2019-11-12,603810,ST丰山,2,2->1,20191112
866559,2019-11-13,603810,ST丰山,2,2->1,20191113


In [392]:
# end with 2 or 4 or 5
st_date2 = st_date.loc[st_date.transition.str.endswith('2')
                       | st_date.transition.str.endswith('4')
                       | st_date.transition.str.endswith('5')]

In [393]:
st_date2_last_day = st_date2.drop_duplicates(subset=['code'], keep='last').copy()
st_date2_last_day['eff_date'] = max_date
st_date2 = pd.concat([st_date2, st_date2_last_day]
                  ).drop_duplicates().sort_values(by=['code', 'eff_date'])


In [394]:
st_date2.drop_duplicates(subset=['code', 'eff_date'], inplace=True)
st_date2['eff_date'] = pd.to_datetime(st_date2['eff_date'])
st_date2 = st_date2.set_index('eff_date').groupby(
    'code').apply(lambda x: x.resample('D').ffill()).reset_index(1).reset_index(drop=True)


In [395]:
st_date2['date'] = st_date2.eff_date.dt.strftime("%Y%m%d")
st_date2.loc[st_date2.date.transform(lambda x: x in CALENDAR_set)]

,eff_date,code,name,status_id,transition,date
0,2000-05-09,3,ST金田A,2,2->4->5,20000509
1,2000-05-10,3,ST金田A,2,2->4->5,20000510
2,2000-05-11,3,ST金田A,2,2->4->5,20000511
3,2000-05-12,3,ST金田A,2,2->4->5,20000512
6,2000-05-15,3,ST金田A,2,2->4->5,20000515
...,...,...,...,...,...,...
1161935,2022-01-27,603996,ST中新,2,2,20220127
1161936,2022-01-28,603996,ST中新,2,2,20220128
1161946,2022-02-07,603996,ST中新,2,2,20220207
1161947,2022-02-08,603996,ST中新,2,2,20220208


In [283]:
st_date['eff_date'] = st_date['eff_date'].astype('string')
st_date = st_date.set_index('eff_date')

In [284]:
st_date

,code,name,status_id
eff_date,,,
20000509,3,ST金田A,2
20010509,3,PT金田A,4
20020614,3,PT金田A,5
19990427,4,ST深安达,2
20010306,4,深安达A,1
...,...,...,...
20210810,603779,威龙股份,1
20190716,603810,ST丰山,2
20191114,603810,丰山集团,1


In [285]:
st_date.index = pd.to_datetime(st_date.index)


In [ ]:
st_date.groupby('code').apply(lambda x: x.resample('D').ffill())


In [296]:
st_date_tmp = st_date.loc[st_date.code == 4]

In [299]:
st_date_tmp.resample('D').ffill()


,code,name,status_id
eff_date,,,
1999-04-27,4,ST深安达,2
1999-04-28,4,ST深安达,2
1999-04-29,4,ST深安达,2
1999-04-30,4,ST深安达,2
1999-05-01,4,ST深安达,2
...,...,...,...
2011-06-04,4,*ST国农,2
2011-06-05,4,*ST国农,2
2011-06-06,4,*ST国农,2


In [240]:
st_date['status_id'] = st_date['status_id'].astype("string")

In [245]:
tmp = st_date.sort_values(by=['code', 'eff_date']).groupby('code').status_id.apply(lambda x: '->'. join(x))

In [244]:
st_date.loc[st_date.code==7]

,code,name,status_id,eff_date
10,7,深达声A,1,20020412
11,7,零七股份,1,20120504
12,7,ST达声,2,20070423
13,7,*ST全新,2,20210430
14,7,ST达声,2,20010419


In [ ]:
# end with 1 or 3
# end with 2
# end with 4 or 5

In [ ]:
tmp.value_counts()

In [264]:
tmp[tmp == '2->4->1->6->7->5']

code
2359      2->4->1->6->7->5
2450      2->4->1->6->7->5
2604      2->4->1->6->7->5
600087    2->4->1->6->7->5
600432    2->4->1->6->7->5
Name: status_id, dtype: object

In [266]:
st_date.loc[st_date.code==2450]

,code,name,status_id,eff_date
1140,2450,ST康得新,2,20190123
1141,2450,康得退,1,20210414
1142,2450,康得退,6,20210414
1143,2450,康得退,7,20210528
1144,2450,康得退,5,20210531
1145,2450,*ST康得,4,20200710


In [415]:
import data_processor as data_processor

In [414]:
data_processor.preprocessHalt('halt_date.csv', 'halt_date.csv')

In [424]:
import importlib
importlib.reload(data_processor)


<module 'data_processor' from 'c:\\Users\\Yao\\source\\repos\\alpharesearch\\data_processor.py'>

In [425]:
data_processor.preprocessHalt('halt_date.csv', 'halt_date.csv')
